In [5]:
import os
import mlflow
import pandas as pd
import psycopg

In [6]:
os.environ["DB_DESTINATION_HOST"] = os.getenv("DB_DESTINATION_HOST")
os.environ["DB_DESTINATION_PORT"] = os.getenv("DB_DESTINATION_PORT")
os.environ["DB_DESTINATION_NAME"] = os.getenv("DB_DESTINATION_NAME")
os.environ["DB_DESTINATION_USER"] = os.getenv("DB_DESTINATION_USER")
os.environ["DB_DESTINATION_PASSWORD"] = os.getenv("DB_DESTINATION_PASSWORD")

In [7]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.environ["DB_DESTINATION_HOST"], 
    "port": os.environ["DB_DESTINATION_PORT"],
    "dbname": os.environ["DB_DESTINATION_NAME"],
    "user": os.environ["DB_DESTINATION_USER"],
    "password": os.environ["DB_DESTINATION_PASSWORD"],
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [8]:
df

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,4815,7270-BDIOA,2017-12-01,2019-10-01,Month-to-month,Yes,Electronic check,90.00,1993.80,Fiber optic,...,No,Yes,Yes,No,Female,0,No,No,Yes,1
1,3860,1732-FEKLD,2015-08-01,NaT,One year,Yes,Bank transfer (automatic),94.75,5121.75,Fiber optic,...,Yes,No,Yes,No,Female,0,No,No,Yes,0
2,1563,8259-NFJTV,2019-07-01,NaT,Month-to-month,No,Mailed check,34.65,246.60,DSL,...,No,Yes,No,No,Female,0,Yes,Yes,None,0
3,1564,3223-DWFIO,2019-10-01,NaT,Month-to-month,No,Electronic check,69.35,261.65,Fiber optic,...,No,No,No,No,Male,1,No,No,No,0
4,1565,2660-EMUBI,2018-12-01,2019-10-01,Month-to-month,Yes,Electronic check,95.35,898.35,Fiber optic,...,No,No,Yes,Yes,Male,1,No,No,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6934,6502-MJQAE,2019-11-01,2019-12-01,Month-to-month,Yes,Electronic check,69.60,69.60,Fiber optic,...,No,No,No,No,Male,0,No,No,No,1
7039,6935,6257-DTAYD,2014-03-01,NaT,Two year,Yes,Credit card (automatic),104.15,7365.30,Fiber optic,...,No,Yes,Yes,Yes,Male,0,Yes,No,Yes,0
7040,6936,4616-ULAOA,2014-09-01,NaT,Two year,Yes,Credit card (automatic),110.80,7245.90,Fiber optic,...,Yes,Yes,Yes,Yes,Female,0,Yes,Yes,Yes,0
7041,6937,7693-LCKZL,2019-05-01,2019-10-01,Month-to-month,Yes,Electronic check,80.15,385.00,Fiber optic,...,No,No,No,No,Male,0,Yes,Yes,Yes,1
